<a href="https://colab.research.google.com/github/thanujaupperi029/GenAi/blob/main/Gen_Ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from scipy.special import softmax
import numpy as np

In [2]:
model_name = "bert-base-cased"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [3]:
mask = tokenizer.mask_token
print(mask)

[MASK]


In [17]:
# Create a sentence with a mask token to be filled in by the model
sentence = f"I am {mask} learning NLP."
# Tokenize the sentence
tokens = tokenizer.tokenize(sentence)
print(tokens)


['I', 'am', '[MASK]', 'learning', 'NL', '##P', '.']


In [ ]:
# Encode the sentence using the tokenizer and return the input tensors
encoded_inputs = tokenizer(sentence, return_tensors='pt')
print(encoded_inputs)

In [19]:
# Get the model's output for the input tensors
outputs = model(**encoded_inputs)
print(outputs)


MaskedLMOutput(loss=None, logits=tensor([[[ -7.1764,  -7.0635,  -7.2361,  ...,  -6.0512,  -5.7479,  -6.1472],
         [ -7.7454,  -8.0146,  -7.8011,  ...,  -6.3420,  -6.2510,  -6.7538],
         [ -7.3258,  -6.9882,  -6.9949,  ...,  -4.2064,  -3.4460,  -4.1748],
         ...,
         [ -6.1488,  -6.4978,  -6.4317,  ...,  -6.4053,  -4.9533,  -5.4914],
         [-14.5795, -14.2772, -14.3355,  ..., -10.6529, -12.4351, -12.0130],
         [-10.3219, -10.9176, -10.5915,  ...,  -8.5259,  -9.5143,  -9.2928]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)


In [20]:
# Detach the logits from the model's output and convert them to numpy arrays
logits = outputs.logits.detach().numpy()[0]
logits.shape

(9, 28996)

In [21]:
len(tokens)

7

In [22]:

# Extract the logits for the mask token
mask_logits = logits[tokens.index(mask) + 1]
print(mask_logits)

[-5.1403375 -5.0165157 -5.220469  ... -5.398006  -4.539516  -5.409568 ]


In [23]:
confidence_scores = softmax(mask_logits)
print(confidence_scores)

[1.7934120e-08 2.0298080e-08 1.6553102e-08 ... 1.3860398e-08 3.2704925e-08
 1.3701060e-08]


In [11]:
confidence_scores.sum()

1.0000001

In [24]:
for i in np.argsort(confidence_scores)[::-1][:5]:
    pred_token = tokenizer.decode(i)
    score = confidence_scores[i]

    # Print the predicted sentence with the mask token replaced by the predicted token, and the confidence score
    print(sentence.replace(mask, pred_token), score)

I am still learning NLP. 0.2071639
I am not learning NLP. 0.16052413
I am now learning NLP. 0.09960015
I am already learning NLP. 0.08688282
I am currently learning NLP. 0.07387184
